In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon

# assumes observations from GBIF and has 'decimalLongitude' and 'decimalLatitude' keys
def create_buffers(obs_df: pd.DataFrame, radius: float) -> gpd.GeoDataFrame:
    """
    Creates 1 km vector buffers around observation points.

    Args:
        obs_df (pd.DataFrame): DataFrame containing GBIF observation coordinates 
                               with 'decimalLongitude' and 'decimalLatitude' columns.

    Returns:
        gpd.GeoDataFrame: A GeoDataFrame with original points and their corresponding 1 km buffers.
    """
    geometry = [Point(lon, lat) for lon, lat in zip(obs_df['decimalLongitude'], obs_df['decimalLatitude'])]
    gdf = gpd.GeoDataFrame(obs_df, geometry=geometry, crs="EPSG:4326")

    gdf_utm = gdf.to_crs("EPSG:32616")
    gdf_utm['buffers'] = gdf_utm.geometry.buffer(radius)
    return gdf_utm

gbif_observations_file_path = '..\\data\\amaranthus_occurrences.csv'
df = pd.read_csv(gbif_observations_file_path)

gdf_utm = create_buffers(df, radius=1000)
gdf_utm.head()

,decimalLongitude,decimalLatitude,eventDate,basisOfRecord,geometry,buffers
0,-90.375983,38.623817,2023-08-29,PRESERVED_SPECIMEN,POINT (206088.199 4280440.95),"POLYGON ((207088.199 4280440.95, 207083.384 42..."
1,-90.898206,38.431925,2022-09-16,PRESERVED_SPECIMEN,POINT (159704.858 4260941.616),"POLYGON ((160704.858 4260941.616, 160700.043 4..."
2,-90.898206,38.431925,2022-09-16,PRESERVED_SPECIMEN,POINT (159704.858 4260941.616),"POLYGON ((160704.858 4260941.616, 160700.043 4..."
3,-90.636111,38.532500,2021-08-21,PRESERVED_SPECIMEN,POINT (183033.318 4271169.926),"POLYGON ((184033.318 4271169.926, 184028.502 4..."
4,-90.635833,38.532408,2020-09-01,PRESERVED_SPECIMEN,POINT (183057.154 4271158.753),"POLYGON ((184057.154 4271158.753, 184052.339 4..."


In [8]:
import numpy as np

def create_control_buffers(bbox: tuple[tuple[int, int], tuple[int, int], tuple[int, int], tuple[int, int]],
                           radius: float) -> gpd.GeoDataFrame:
    """
    Generates random control buffers with a 1km radius within a bounding box.

    Args:
        bbox (tuple): Bounding box coordinates (min_x, min_y, max_x, max_y).

    Returns:
        gpd.GeoDataFrame: A GeoDataFrame with random points and their corresponding 1 km buffers.
    """
    illinois_bbox = Polygon(bbox)
    min_x, min_y, max_x, max_y = illinois_bbox.bounds

    np.random.seed(42)

    n_controls = len(gdf_utm)
    random_points = [
        Point(np.random.uniform(min_x, max_x),
            np.random.uniform(min_y, max_y))
        for _ in range(n_controls)
    ]
    gdf_controls = gpd.GeoDataFrame(geometry=random_points, crs="EPSG:4326")
    gdf_controls['buffers'] = gdf_controls.to_crs("EPSG:32616").buffer(radius)

    return gdf_controls


il_bbox = ((-91.513, 36.970), (-87.495, 36.970), (-87.495, 42.508), (-91.513, 42.508), (-91.513, 36.970))
gdf_controls = create_control_buffers(il_bbox, radius=1000)
gdf_controls.head()

,geometry,buffers
0,POINT (-90.0081 42.23506),"POLYGON ((252784.252 4680256.624, 252779.437 4..."
1,POINT (-88.57185 40.28537),"POLYGON ((367384.919 4460616.593, 367380.103 4..."
2,POINT (-90.88612 37.8339),"POLYGON ((158975.716 4194506.59, 158970.901 41..."
3,POINT (-91.27962 41.76688),"POLYGON ((145243.706 4632752.902, 145238.891 4..."
4,POINT (-89.09772 40.89131),"POLYGON ((324283.319 4528809.321, 324278.504 4..."


In [ ]:
import zipfile
import os

# zip_file_path = "../_NLCD_519W897hevlAKeQOwMVE.zip"
# extract_dir = "../nlcd_data"
# os.makedirs(extract_dir, exist_ok=True)

# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_dir)


In [9]:
# extract_dir = "..\us_il_shapefiles"
# zip_file_path = os.path.join(extract_dir, "IL_BNDY_State.zip")

# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_dir)

# shapefile_path = os.path.join(extract_dir, "IL_BNDY_State_Ln.shp")
# il_gdf = gpd.read_file(shapefile_path)

#for more precise illinois state boundaries--didn't end up using

il_coordinates = [
    (-91.5136, 37.3957),
    (-87.5, 37.3957), 
    (-87.5, 42.5),
    (-89.5, 42.5),
    (-90.5, 42.3),
    (-91.5, 42.3),
    (-91.5136, 37.3957)   
]

il_polygon = Polygon(il_coordinates)
il_gdf = gpd.GeoDataFrame({'geometry': [il_polygon]})
il_gdf = il_gdf.set_crs("EPSG:4326")

In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject

    # this function is unecessary because we should be reprojecting
    # the vectors onto the raster NLCD data, as it is in the Albers Equal-Area 
    # conic projection, while WGS-84 distorts area
    # also saves from having to store two versions of the raster

def reproject_raster(input_path: str, output_path: str, dst_crs: str):
    """
    Reprojects a raster file to a specified coordinate reference system (CRS).

    Args:
        input_path (str): Path to the input raster file.
        output_path (str): Path to save the reprojected raster file.
        dst_crs (str): The target coordinate reference system in PROJ or EPSG format (e.g., 'EPSG:4326').

    Returns:
        None
    """
    with rasterio.open(input_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds
        )
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open(output_path, 'w', **kwargs) as dst:
            reproject(
                source=rasterio.band(src, 1),
                destination=rasterio.band(dst, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=rasterio.enums.Resampling.nearest
            )

dst_crs = ""
extract_dir = "..\processed_rasters"
in_tiff_file_path = '..\\nlcd_data\\Annual_NLCD_LndCov_2023_CU_C1V0_519W897hevlAKeQOwMVE.tiff'
out_tif_file_path = os.path.join(extract_dir, "IL_NLCD_2023_WGS84.tiff")
# reproject_raster(in_tiff_file_path, out_tif_file_path, dst_crs=dst_crs)


In [13]:
import rasterio
from rasterio.crs import CRS

# define custom CRS
nlcd_custom_wkt = """
PROJCS["AEA WGS84",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0],
        UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Albers_Conic_Equal_Area"],
    PARAMETER["latitude_of_center",23],
    PARAMETER["longitude_of_center",-96],
    PARAMETER["standard_parallel_1",29.5],
    PARAMETER["standard_parallel_2",45.5],
    PARAMETER["false_easting",0],
    PARAMETER["false_northing",0],
    UNIT["metre",1,AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],AXIS["Northing",NORTH]]
"""
custom_crs = rasterio.crs.CRS.from_wkt(nlcd_custom_wkt)
gdf_aea = gdf_utm.to_crs(custom_crs)
gdf_controls_aea = gdf_controls.to_crs(custom_crs)

print(gdf_aea['buffers'].crs)

PROJCS["AEA WGS84",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


In [14]:
nlcd_classes = {
    11: "Water",
    12: "Perennial Ice/Snow",
    21: "Developed, Open Space",
    22: "Developed, Low Intensity",
    23: "Developed, Medium Intensity",
    24: "Developed, High Intensity",
    31: "Barren Land",
    41: "Deciduous Forest",
    42: "Evergreen Forest",
    43: "Mixed Forest",
    52: "Shrub/Scrub",
    71: "Grassland/Herbaceous",
    81: "Pasture/Hay",
    82: "Cultivated Crops",
    90: "Woody Wetlands",
    95: "Emergent Herbaceous Wetlands"
}

In [ ]:
from rasterstats import zonal_stats
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))

from diversity_indices import calculate_shannon_index

def nlcd_zonal_stats(geometry: gpd.GeoSeries | gpd.GeoDataFrame, nlcd_file_path) -> list[dict[str, float | str]]:
    """
    Computes land cover class proportions within given geometries using NLCD raster data.

    Args:
        geometry (gpd.GeoSeries | gpd.GeoDataFrame): Geospatial data representing areas of interest.

    Returns:
        list[dict[str, float | str]]: A list of dictionaries containing land cover class proportions 
                                      and Shannon diversity index for each geometry.
    """
   
    result = []

    # uses the vector buffers as overlays and calculates the proportion of each
    # class in the raster National Land Cover data
    stats = zonal_stats(
        geometry, 
        nlcd_file_path, 
        categorical=True,
        category_map=nlcd_classes,
        geojson_out=True,
        all_touched = False
    )
    for zone in stats:
        props = zone['properties'] #dictionary corresponding to number of pixels per class
        total = sum(props.values())  #total pixels in buffer
        percentages = {k: (v / total) * 100 for k, v in props.items() if k != 'id'}
        percentages['id'] = zone['id']

        
        proportions = np.array(list(percentages.values()))
        shannon_index = calculate_shannon_index(proportions)
        percentages['shannon_index'] = shannon_index
        
        result.append(percentages)

    return result

In [ ]:
def mean_clean_df(input_results: list[dict[str, float|str]]) -> pd.Series:
    """
    Cleans dataframes and computes the mean proportions of land cover classes and the mean Shannon index.

    Args:
        input_results (list[dict[str, float | str]]): List of dictionaries with land cover proportions 
                                                      and Shannon diversity index.

    Returns:
        pd.Series: Mean proportions of each land cover class.
        float: Mean Shannon diversity index.
    """
    data_frame = pd.DataFrame(input_results).fillna(0)
    if 'id' in data_frame.columns:
        data_frame = data_frame.drop(columns=['id'])
    data_frame = data_frame.apply(pd.to_numeric, errors='coerce')
    
    # get the mean shannon index over all the buffers, then drop that column
    mean_shannon_index = data_frame['shannon_index'].mean()
    data_frame = data_frame.drop(columns=['shannon_index'])
    
    # sort alphabetically by land use class
    data_frame = data_frame.sort_index(axis=1)
    return data_frame.mean(), mean_shannon_index

def out_df(geometry: gpd.GeoSeries | gpd.GeoDataFrame, nlcd_file_path: str) -> pd.DataFrame:
    """
    Generates a DataFrame of land cover class proportions and computes the mean Shannon index.

    Args:
        geometry (gpd.GeoSeries | gpd.GeoDataFrame): Geospatial data to analyze.

    Returns:
        pd.DataFrame: DataFrame containing land cover class names and their average proportions.
        float: Mean Shannon diversity index across all geometries.
    """
    vector = geometry.to_crs(custom_crs)
    result = nlcd_zonal_stats(vector, nlcd_file_path)
    result_series, mean_shannon_index = mean_clean_df(result)
    result_df = pd.DataFrame({'lu_class':result_series.index, 'proportions':result_series.values})
    return result_df, mean_shannon_index

results_df, waterhemp_shannon_idx = out_df(gdf_aea['buffers'], "..\\nlcd_data\\Annual_NLCD_LndCov_2023_CU_C1V0.tif")
control_results_df, control_shannon_idx = out_df(gdf_controls['buffers'], "..\\nlcd_data\\Annual_NLCD_LndCov_2023_CU_C1V0.tif")
il_results_df, _ = out_df(il_gdf, "..\\nlcd_data\\Annual_NLCD_LndCov_2023_CU_C1V0.tif")
print(results_df)
print(f"Shannon diversity index of land within 1000m of waterhemp occurences: {waterhemp_shannon_idx},\n\
Control Shannon diversity index: {control_shannon_idx}")

results_df.to_csv("..\\amaranthus_results.csv", index=False)
control_results_df.to_csv("..\\control_results.csv", index=False)
il_results_df.to_csv("..\\il_results.csv", index=False)


                        lu_class  proportions
0                    Barren Land     1.002892
1               Cultivated Crops    24.628635
2               Deciduous Forest    21.650435
3      Developed, High Intensity     2.809434
4       Developed, Low Intensity     6.973916
5    Developed, Medium Intensity     5.426161
6          Developed, Open Space     7.544756
7   Emergent Herbaceous Wetlands     1.542646
8               Evergreen Forest     0.358571
9           Grassland/Herbaceous     0.395297
10                  Mixed Forest     1.629594
11                   Pasture/Hay     8.620570
12                   Shrub/Scrub     0.047466
13                         Water     9.888547
14                Woody Wetlands     7.481081
Shannon diversity index of land within 1000m of waterhemp occurences: 1.6325656543760083,
Control Shannon diversity index: 1.205654308968665


In [ ]:
# entire workflow for gbif_data

def add_observation_land_use_data():
    df = pd.read_csv('..\\data\\0001075-250213122211068.csv', delimiter='	')
    filtered_df = df.dropna(subset=['decimalLongitude', 'decimalLatitude'])
    filtered_df = filtered_df[filtered_df["year"] < 2024]
    gdf = create_buffers(filtered_df, radius=300)
    gdf = gdf.to_crs(custom_crs)
    #gdf = gdf.reset_index(drop=True)

    new_columns = ["land_cover_class", "top_class_300m", "proportion_agricultural_300m", "shannon_idx_300m"]
    for col in new_columns:
        filtered_df[col] = None

    for idx, row in gdf.iterrows():
        raster_path = f"..\\nlcd_data\\Annual_NLCD_LndCov_{row["year"]}_CU_C1V0.tif"
        point = gdf.loc[[idx]].geometry.iloc[0]
        with rasterio.open(raster_path) as src:
            coord = [(point.x, point.y)]
            pixel_value = next(src.sample(coord))[0]
            # land_class = nlcd_classes.get(pixel_value, None)
            # row_gdf = gpd.GeoDataFrame([row], columns=gdf.columns, crs=gdf.crs)
            # buffer_geometry = row['buffers']
            # buffer_gdf = gpd.GeoDataFrame({'geometry': [buffer_geometry]}, index=[idx])
            # buffer_gdf.set_crs(custom_crs, allow_override=True, inplace=True)                

            #proportions = nlcd_zonal_stats(gpd.GeoSeries(row_gdf['buffers'], crs=row_gdf.crs), raster_path)[0]
            #print(proportions)

        filtered_df.at[idx, 'land_cover_class'] = nlcd_classes.get(pixel_value, None)
    df_combined = df.reindex(filtered_df.index)
    df_combined['land_cover_class'] = filtered_df['land_cover_class']


    print(df_combined['land_cover_class'].value_counts())
    df_combined.to_csv('..\\amaranthus_land_classes')

add_observation_land_use_data()


land_cover_class
Developed, Low Intensity        74
Cultivated Crops                71
Deciduous Forest                63
Woody Wetlands                  60
Developed, Open Space           50
Pasture/Hay                     46
Developed, Medium Intensity     41
Emergent Herbaceous Wetlands    39
Water                           31
Grassland/Herbaceous            29
Developed, High Intensity       17
Barren Land                     15
Evergreen Forest                 8
Shrub/Scrub                      2
Mixed Forest                     1
Name: count, dtype: int64
